# CDS303 Fall 2023 - Final Project
Group 3:
- Jose Rodriguez
- Andrew Lee
- Elene Lipartiani
- Leul Kifle
- Fariha Askar

## Introduction


**IPO** stands for initial public offering, and refers to the process of a private corporation offering their shares to the public through a new stock issuance. This means transitioning ownership, which is why the IPO process can be known as “going public”. IPOs are important since they provide companies with an opportunity to get capital by offering their shares through the primary market. By selling shares to the public, the company can raise funds that can be used for various purposes, such as expanding operations, paying off debt, or funding research and development. The process of going public involves many regulatory requirements, some of which are done through the Securities and Exchange Commission (SEC) in the U.S. Once the IPO process is completed, the company's shares are listed on a stock exchange. One example of this is the New York Stock Exchange (NYSE) or NASDAQ. This listing allows the shares to be traded among investors in the secondary market. The price of the shares fluctuates based on the supply and demand in the open market and not all IPOs result in a successful investment, which is why analyzing their recent performance is important for determining good investment opportunities.

**IPOs can be interesting to study and invest in due to their growth potential, access to new opportunities, liquidity, and potential for early gains**. However, they also come with risks and challenges, and investors should conduct thorough research and consider their risk tolerance before participating in IPOs. **In this notebook, we will explore the viability of investing in companies that have gone public  recently using the Cross-Industry Standard Process for Data Mining (CRISP-DM) methodology.**

In [35]:
import warnings
warnings.filterwarnings("ignore")

## I. Business Understanding

### Business Problem

How can we predict the best and worst investment opportunities for tech companies that went public recently? We are a GMU student organization with money available to invest. We have a preference for Technology companies that have gone public recently (Initial Public Offering Company or IPOC), and we would like to perform **daily trading** of their shares. Therefore, **we need to identify what are the best and worst investment opportunities for the following day**. To determine whether the companies are good investment opportunities, we will be analyzing the performance of their recent IPO.

### Analytical Problem

We need **to predict the daily share price % change of IPOCs**. If the future daily price goes up, it might be an opportunity to keep our current number of shares or decide to get more shares of that company. Otherwise, we will sell our shares to recover our investment and profit. Since we are predicting the daily share price, this will be a regression problem. To predict it, we **will use time series algorithms** and since we have access to historic share prices, we **will use a supervised learning approach**.

### Initial Public Offering Companies

**NASDAQ**, which stands for the National Association of Securities Dealers Automated Quotations, is one of the two major stock exchanges in the United States. We have selected 5 companies that have gone public recently. Company names and initial IPO dates are described below:

1. Asset Entity - **ASST** (2/3/2023)
2. Snail Inc. - **SNAL** (11/10/2022)
3. GigaCloud - **GCT** (8/13/2022)
4. Mobile Global Esports - **MGAM** (7/29/2022)
5. Knightscope - **KSCP** (1/27/2022)

In [1]:
# creating a List with ticker names of companies
companies = ['GCT','MGAM','ASST','SNAL','KSCP']

## II. Data Understanding

### Required Features Description

There are many variables involved in evaluating the performance of an IPO. Since we are interested in predicting whether the companies are good investment opportunities, we will be looking at the **share price % change of previous days**, **news sentiment scores** and **sale volume** to predict the **forecasted share price % change** and thus determine investment viability. 

The descriptions for our features are the following:

- Forecasted Share Price % Change (**Depedent/Response variable**): This feature describes the anticipated or projected percentage change in the share price of the IPO in the future. It will be determined by inputting all of the independent variables we covered above into our
model, for each of the 5 recent tech companies which we’ve selected.

- % Change of Previous Day Share Price (**Independent/Explanatory variable**): This feature describes the percentage change in the price
of IPO shares by comparing the current price (or closing price) of a share to a previous price. A positive value for the % change would indicate an increase in the share price compared to the previous price, while a negative value would indicate a decrease in the share price compared to the previous one. This is particularly valuable for assessing short-term performance of IPO shares and helps determine investment viability since it tracks the movement of the share price.

- News Sentiment Score (**Independent/Explanatory variable**): This feature describes the sentiment around an IPO and company going public, including market sentiment, investor sentiment, and public perception. Our study will assess sentiment score based on media coverage by news articles and press releases about our selected IPOs and assign them to categories ranging from Bullish (indicating a positive outlook about the IPOs shares by investors) to Bearish (indicating a pessimistic or skeptical outlook about the IPO shares by investors). These designations are an important part of assessing investment opportunities since Bullish sentiment can indicate a higher initial offering price and strong demand for shares, compared to the lower pricing and weaker demand indicated by Bullish sentiment, thus providing perspective on whether an IPO may be considered a good investment opportunity.

    Below, we’ve included a chart showing the sentiment score range of values as relevant to their sentiment labels:

    | Range           | Sentiment Score      |
    |-----------------|-----------------------|
    | x ≤ -0.35       | Bearish               |
    | -0.35 < x ≤ -0.15 | Somewhat-Bearish    |
    | -0.15 < x < 0.15  | Neutral             |
    | 0.15 ≤ x < 0.35   | Somewhat Bullish    |
    | x ≥ 0.35         | Bullish               |

    Table 1: Sentiment Score Interpretation


- Sales Volume (**Independent/Explanatory variable**): This feature describes the total number of shares of a company’s stock that have been bought and sold on a stock exchange within a specific time frame. This is a key factor in determining IPO performance since it can indicate how actively investors are trading the stock and is used alongside other metrics to assess the market sentiment around IPOs, therefore making it an important way to determine investment viability

Below, there is a representation of the interaction of our variables using a **black box** model:

<figure>
  <img src="BlackBoxModel-Group3.png">
  <figcaption>Figure 1. Relationship between Response and Explanatory variables.</figcaption>
</figure>

### Data Sets

For the purpose of this project, we will use 2 main sources of data: [Federal Reserve Bank of San Francisco](https://www.frbsf.org/economic-research/indicators-data/daily-news-sentiment-index/) and [Alpha Vantage Api](https://www.alphavantage.co/documentation/).

It is important to note that an API Key is required to interact with the API service provided by Alpha Vantage. A free API key can be obtained [here](https://www.alphavantage.co/support/#api-key).

In [2]:
# Crating variable to store API key
api_key = 'ALORD0KVCFQTZ03M'

#### a) Daily Shares Prices and Sales Volume

We will use the [**TIME_SERIES_DAILY**](https://www.alphavantage.co/documentation/#daily) api to get context of the daily share prices and sales volume. By default, this API function returns 100 data points. We will set the **outputsize** parameter to **full** so it returns the full-length time series of 20+ years of historical data.

In [215]:
# Library to make HTTP request
import requests
# Library for data manipulation
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [15]:
# Creating empty dataframe
dailyPricesData = pd.DataFrame()
for ticker in companies:
    # Getting data for each ticker
    dailyDataTicker = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={api_key}&outputsize=full").json()
    # Converting data in a Data Frame
    dailyDataTicker = pd.DataFrame(dailyDataTicker['Time Series (Daily)']).transpose().reset_index().rename(columns={'index': 'date'})
    # Adding column with Ticker name
    dailyDataTicker['TickerName'] = ticker
    # Adding Data Frame to monthly data
    dailyPricesData = pd.concat([dailyPricesData, dailyDataTicker], ignore_index=True)

After retrieving data using the API (November 04, 2023), we were able to obtain **1,509** observations for 7 features.

In [16]:
# Getting dimensions of our dataframe
dailyPricesData.shape

(1509, 7)

In the cell below, we can visualize the first 5 observations for the data retrieved using the API.

In [17]:
# Visualizing the dataframe
dailyPricesData.head(n = 5)

,date,1. open,2. high,3. low,4. close,5. volume,TickerName
0,2023-11-03,10.6600,11.1700,10.4100,10.7900,655003,GCT
1,2023-11-02,10.1100,10.6000,9.7600,10.6000,551432,GCT
2,2023-11-01,9.6600,10.0899,9.3200,9.9200,869370,GCT
3,2023-10-31,8.8300,9.5500,8.7500,9.4700,1290602,GCT
4,2023-10-30,8.2800,9.0000,8.2800,8.8900,452530,GCT


Let's check the number of observations per TickerName (Compnay abbreviation): Asset Entity - ASST is the company with the minimum number of observations, 190. This is a good reference for our project because, in general, at least 50 data points are required when trying to develop a Time Series model.

In [18]:
dailyPricesData.value_counts('TickerName')

TickerName
KSCP    446
MGAM    320
GCT     306
SNAL    247
ASST    190
dtype: int64

From this dataframe, we will use the features **date**, **4.close**, **5. volume**, and **TickerName**. Even though we haven't started the **Data Preparation** phase of our project, we will modify the names of these features to facilitate our analysis.

In [19]:
# Dropping not necessary columns
dailyPricesData.drop(columns = ['1. open','2. high','3. low'], inplace = True)
# Renaming columns
dailyPricesData.rename(columns = {'date':'day','4. close':'closingPrice','5. volume':'salesVolume','TickerName':'company'}, inplace = True)
dailyPricesData


,day,closingPrice,salesVolume,company
0,2023-11-03,10.7900,655003,GCT
1,2023-11-02,10.6000,551432,GCT
2,2023-11-01,9.9200,869370,GCT
3,2023-10-31,9.4700,1290602,GCT
4,2023-10-30,8.8900,452530,GCT
...,...,...,...,...
1504,2022-02-02,9.5500,12613046,KSCP
1505,2022-02-01,12.4400,18493106,KSCP
1506,2022-01-31,21.4000,42899912,KSCP
1507,2022-01-28,16.2900,26574308,KSCP


In order to expedite our analysis, we will store all the data retrieved in a csv file.

In [20]:
# Saving dataframe as csv file
dailyPricesData.to_csv('DailyPricesIpoCompaniesNov04.csv', index=False)

#### b) News Sentiment



We will use the [**NEWS_SENTIMENT**](https://www.alphavantage.co/documentation/#news-sentiment) api to get news sentiment context. By default, this API function returns 50 data points. We will set the **limit** parameter to **1000**, the maximum number of data points that can be retrieved when using a free API key. It is important to note that this API provides data for 15 different topics:

- Blockchain: blockchain
- Earnings: earnings
- IPO: ipo
- Mergers & Acquisitions: mergers_and_acquisitions
- Financial Markets: financial_markets
- Economy - Fiscal Policy (e.g., tax reform, government spending): economy_fiscal
- Economy - Monetary Policy (e.g., interest rates, inflation): economy_monetary
- Economy - Macro/Overall: economy_macro
- Energy & Transportation: energy_transportation
- Finance: finance
- Life Sciences: life_sciences
- Manufacturing: manufacturing
- Real Estate & Construction: real_estate
- Retail & Wholesale: retail_wholesale
- Technology: technology

In [21]:
sentimentTopics = ['blockchain','earnings','ipo','mergers_and_acquisitions','financial_markets',
                   'economy_fiscal','economy_monetary','economy_macro','energy_transportation',
                   'finance','life_sciences','manufacturing','real_estate','retail_wholesale',
                   'technology']

Each topic will require us to make an API call. Therefore, we need 15 API calls per compnay. Since we are using a free API key, we can only make 5 API calls witin a minute, and a total of 25 API call per day. We got a premium API key for one month to retrieve 30 API calls per minute and no limit per day. We still need to wait for one minute after 30 API calls. So, we will use the **time** python library to wait for 60 seconds at least before executing the code per company.

In [4]:
# Library to manage execution time
import time

In [ ]:
# Creating empty list
sentimentData = list()
# Getting News & Sentiment for compnaies
for ticker in companies:
    # Looping through all topics
    for topic in sentimentTopics:
        sentimentTopic = requests.get(f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={api_key}&limit=1000&sort=LATEST&topics={topic}').json()
        # Adding dictionary to list
        sentimentData.append(sentimentTopic)
    # Waiting for one minute before calling API for next company
    time.sleep(65)
# Creating data frame with list
sentimentNews = pd.DataFrame(sentimentData)

In the cell below, we can visualize the first 5 observations for the data retrieved using the API.

In [71]:
# Visualizing the dataframe
sentimentNews.head(n = 5)

,items,sentiment_score_definition,relevance_score_definition,feed
0,7,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",[{'title': 'DBX: 3 Top Tech Stocks Positioned ...
1,86,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",[{'title': 'Telus International ( TIXT ) Q3 ...
2,61,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",[{'title': 'GIGACLOUD ALERT: Bragar Eagel & Sq...
3,0,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",[]
4,0,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",[]


Each row of our dataframe describes all the articles for a compnay/topic. We need to extract this information and create a dataframe where every row describes an article for a compnay.

In [72]:
# Removing companies/topics without articles
sentimentNews = sentimentNews[sentimentNews['items'] != '0']
# For each line, feed has a list of dictionaries.
# Each dicitonary represents an article. We will create a row per article
sentimentNews = sentimentNews.explode('feed')
sentimentNews.reset_index(drop=True,inplace=True)
# The key of each dictionary describes a feature. We will create a new column for each dictionary key
feedData = pd.json_normalize(sentimentNews['feed'])
sentimentNews = pd.concat([sentimentNews,feedData], axis=1).drop(['feed','items',
                                                                  'sentiment_score_definition',
                                                                  'relevance_score_definition'], axis=1)
# Even though the API provides us with an overall sentiment score and label columns,
# column ticker_sentiment contains a list of dictionaries with sentiment scores and labels per company.
# First, we will create a column to record the number of companies related to the article
sentimentNews['numberOfCompanies'] = sentimentNews.apply(lambda row: len(row['ticker_sentiment']), axis=1)
# Creating a row per company (Dictionary)
sentimentNews = sentimentNews.explode('ticker_sentiment')
sentimentNews.reset_index(drop=True,inplace=True)
# Expanding keys of dictionaries
tickerSentimentData = pd.json_normalize(sentimentNews['ticker_sentiment'])
sentimentNews = pd.concat([sentimentNews,tickerSentimentData], axis=1).drop(['ticker_sentiment'], axis=1)
sentimentNews = sentimentNews.rename(columns={'ticker':'company',
                                              'relevance_score':'companyRelevanceScore',
                                              'ticker_sentiment_score':'companySentimentScore',
                                              'ticker_sentiment_label':'companySentimentLabel'})
# Each article is related to different topics
# Let's create a row per topic
sentimentNews = sentimentNews.explode('topics')
sentimentNews.reset_index(drop=True,inplace=True)
# Let's create a column for the topic and topic relevance score
topicData = pd.json_normalize(sentimentNews['topics'])
sentimentNews = pd.concat([sentimentNews,topicData], axis=1).drop(['topics','authors'], axis=1)
sentimentNews = sentimentNews.rename(columns={'relevance_score':'topicRelevanceScore'})
# Since more than one company can be related to the article, let's remove duplicate rows
sentimentNews = sentimentNews.drop_duplicates()
sentimentNews.reset_index(drop=True,inplace=True)

Let's visualize the first 5 rows of our dataframe again:

In [73]:
# Visualizing the dataframe
sentimentNews.head(n = 5)

,title,url,time_published,summary,banner_image,source,category_within_source,source_domain,overall_sentiment_score,overall_sentiment_label,numberOfCompanies,company,companyRelevanceScore,companySentimentScore,companySentimentLabel,topic,topicRelevanceScore
0,DBX: 3 Top Tech Stocks Positioned for Profitab...,https://stocknews.com/news/dbx-saic-gct-3-top-...,20231031T182138,"Despite macroeconomic challenges, the tech ind...",https://stocknews.com/wp-content/uploads/2021/...,Stocknews.com,n/a,stocknews.com,0.287641,Somewhat-Bullish,3,GCT,0.169477,0.234357,Somewhat-Bullish,Financial Markets,0.996023
1,DBX: 3 Top Tech Stocks Positioned for Profitab...,https://stocknews.com/news/dbx-saic-gct-3-top-...,20231031T182138,"Despite macroeconomic challenges, the tech ind...",https://stocknews.com/wp-content/uploads/2021/...,Stocknews.com,n/a,stocknews.com,0.287641,Somewhat-Bullish,3,GCT,0.169477,0.234357,Somewhat-Bullish,Earnings,0.999993
2,DBX: 3 Top Tech Stocks Positioned for Profitab...,https://stocknews.com/news/dbx-saic-gct-3-top-...,20231031T182138,"Despite macroeconomic challenges, the tech ind...",https://stocknews.com/wp-content/uploads/2021/...,Stocknews.com,n/a,stocknews.com,0.287641,Somewhat-Bullish,3,GCT,0.169477,0.234357,Somewhat-Bullish,Technology,1.0
3,DBX: 3 Top Tech Stocks Positioned for Profitab...,https://stocknews.com/news/dbx-saic-gct-3-top-...,20231031T182138,"Despite macroeconomic challenges, the tech ind...",https://stocknews.com/wp-content/uploads/2021/...,Stocknews.com,n/a,stocknews.com,0.287641,Somewhat-Bullish,3,GCT,0.169477,0.234357,Somewhat-Bullish,Blockchain,0.158519
4,DBX: 3 Top Tech Stocks Positioned for Profitab...,https://stocknews.com/news/dbx-saic-gct-3-top-...,20231031T182138,"Despite macroeconomic challenges, the tech ind...",https://stocknews.com/wp-content/uploads/2021/...,Stocknews.com,n/a,stocknews.com,0.287641,Somewhat-Bullish,3,SAIC,0.141559,0.168108,Somewhat-Bullish,Financial Markets,0.996023


From this dataframe, we will use the features **time_published**, **company**, **companyRelevanceScore**, and **companySentimentScore**. Even though we haven't started the **Data Preparation** phase of our project, we will filter company names and count the number of articles per company

In [75]:
# Filtering IPO companies
sentimentNews = sentimentNews[sentimentNews['company'].isin(companies)]
# Filtering relevant features
sentimentNews = sentimentNews[['time_published','company','companyRelevanceScore','companySentimentScore']]
# Number of articles per company
sentimentNews.value_counts('company')

company
GCT     490
ASST    151
KSCP    139
MGAM    113
SNAL     43
dtype: int64

Even though we have retrieved a lot of articles per company, it is important to remember that our analysis is per day. Therefore, a more relevant summary statistic would be the number of days where the company has at least one article. To got that statistic, we will use the **time_published** [String] feature to get the **day** of publication. We will create a new variable, day, and recalculate the summary of statistics.

In [ ]:
# Updating publication datetime format
sentimentNews['time_published'] = pd.to_datetime(sentimentNews['time_published'], format='%Y%m%dT%H%M%S')
# Getting publication day
sentimentNews['Day'] = sentimentNews['time_published'].dt.date
# Dropping unnecessary column
sentimentNews.drop(columns = 'time_published', inplace = True)
# Resetting dataframe index
sentimentNews.reset_index(drop = True, inplace = True)

Now, let's count the number of days with sentiment news data available per company.

In [87]:
# Counting number of days with available data
count_per_company = sentimentNews.groupby('company')['Day'].nunique().reset_index().sort_values(by = 'Day', ascending = False)
print(count_per_company)

  company  Day
1     GCT   71
0    ASST   27
2    KSCP   22
3    MGAM   12
4    SNAL    7


The results showed above were not expected. If we compare the number of data points here with the number of data points retrieved for daily prices and sales volume, there will be a lot of days with null values when aggregating the information. Therefore, **we need to use a different source of data for sentiment data**.

For the purpose of this project, we have decided to use the **daily news sentiment index** from the [Federal Reserve Bank of San Francisco](https://www.frbsf.org/economic-research/indicators-data/daily-news-sentiment-index/). After reviewing this website, we had access to an [excel file](https://www.frbsf.org/wp-content/uploads/sites/4/news_sentiment_data.xlsx?20231030) with daily data for every trading day since 1980 until oct 15 of 2023.

In [92]:
# Reading excel file
dailyNewsSentimentIndex = pd.read_excel('news_sentiment_data.xlsx', sheet_name = 'Data')

Let's visualize the first 5 rows of our dataframe again:

In [93]:
# Visualizing the dataframe
dailyNewsSentimentIndex.head(n = 5)

,date,News Sentiment
0,1980-01-01,-0.038265
1,1980-01-02,-0.107314
2,1980-01-03,-0.090467
3,1980-01-05,-0.068480
4,1980-01-06,-0.090640


### Exploratory Analysis of Features

Let's start by reading the **CSV files** we downloaded before:


In [5]:
# Reading CSV files
dailyPricesData = pd.read_csv('DailyPricesIpoCompaniesNov04.csv')
dailyNewsSentimentIndex = pd.read_excel('news_sentiment_data.xlsx', sheet_name = 'Data')
# Visualizing frist 5 rows of data of each dataset
print('Daily Share Prices and Sales Volume:')
print(dailyPricesData.head (n = 5))
print('\nDaily News Sentiment Index:')
print(dailyNewsSentimentIndex.head (n = 5))

Daily Share Prices and Sales Volume:
          day  closingPrice  salesVolume company
0  2023-11-03         10.79       655003     GCT
1  2023-11-02         10.60       551432     GCT
2  2023-11-01          9.92       869370     GCT
3  2023-10-31          9.47      1290602     GCT
4  2023-10-30          8.89       452530     GCT

Daily News Sentiment Index:
        date  News Sentiment
0 1980-01-01       -0.038265
1 1980-01-02       -0.107314
2 1980-01-03       -0.090467
3 1980-01-05       -0.068480
4 1980-01-06       -0.090640


Before we start exploring our data, we will consolidate both datasets by adding news sentiment context to each row of the dailyPricesData dataframe. To execute this operation, we will need to convert the data type of **day** in dailyPricesData. Finally, we will filter out any day after October 15, 2023.

In [6]:
# Updating data type for day
dailyPricesData['day'] = pd.to_datetime(dailyPricesData['day'], format='%Y-%m-%d')
# Let's add News Sentiment context
dailyData = dailyPricesData.merge(dailyNewsSentimentIndex, left_on=['day'], right_on=['date'], how='left')
dailyData.drop(columns = 'date', inplace = True)
# Filtering out days after October 15, 2023
dailyData = dailyData[dailyData['day'] <= '2023-10-15']
# Resetting Index
dailyData.reset_index(drop = True, inplace = True)

Let's visualize the first 5 rows of the new dataframe:

In [7]:
# Visualizing dataframe
dailyData.head()

,day,closingPrice,salesVolume,company,News Sentiment
0,2023-10-13,8.69,476329,GCT,-0.073652
1,2023-10-12,9.19,329084,GCT,-0.073453
2,2023-10-11,8.77,427515,GCT,-0.084724
3,2023-10-10,9.07,1266355,GCT,-0.107545
4,2023-10-09,9.96,514556,GCT,-0.079329


#### a) Univariate Analysis

- **Data Type**

Lets start by recognizing the type of data identified by Pandas. As we can see in the cell below, closingPrice and salesVolume were identified as numerical varaibles. On the other hand, day and company were identified as objects. _**Data Transformation required**_: We need to transform company to a not ordered category.

In [8]:
dailyData.dtypes

day               datetime64[ns]
closingPrice             float64
salesVolume                int64
company                   object
News Sentiment           float64
dtype: object

- **Null Values**

As you can see in the cell below, **there are not null values for all our features**.

In [9]:
dailyData.isnull().sum()

day               0
closingPrice      0
salesVolume       0
company           0
News Sentiment    0
dtype: int64

- **Summary of Statistics**

########### Add description for summary of each variable. We need to talk about the different scales

In [10]:
dailyData.describe(include = 'all',datetime_is_numeric = True)

,day,closingPrice,salesVolume,company,News Sentiment
count,1434,1434.000000,1.434000e+03,1434,1434.000000
unique,NaN,NaN,NaN,5,NaN
top,NaN,NaN,NaN,KSCP,NaN
freq,NaN,NaN,NaN,431,NaN
mean,2023-03-01 22:18:34.644351488,3.134253,1.248876e+06,NaN,-0.135592
min,2022-01-27 00:00:00,0.282000,2.440000e+02,NaN,-0.429641
25%,2022-11-18 18:00:00,0.949600,5.763100e+04,NaN,-0.204004
50%,2023-03-21 00:00:00,1.650000,1.665150e+05,NaN,-0.108062
75%,2023-07-05 00:00:00,4.277500,5.864898e+05,NaN,-0.048407
max,2023-10-13 00:00:00,48.010000,9.025801e+07,NaN,0.034715


- **Distribution & Tendency of Values - Closing Price**

############ Add description of histogram and scatter plot per company, plot will be dynamic and you can filter per company

        - Company 1
        - Company 2
        - Company 3
        - Company 4
        - Company 5

In [11]:
# Library for Visualizations
import plotly.express as px

In [12]:
# Checking distribution for closing price
histClosingPrice = px.histogram(dailyData,
                   x='closingPrice',
                   color='company',
                   marginal='box')
histClosingPrice.update_layout(title_text="Distribution of Closing Price - Per Company")
histClosingPrice.show()

In [13]:
# Checking tendency for closing price
tendencyClosingPrice = px.scatter(dailyData,
                   x='day',y='closingPrice',
                   color='company')
tendencyClosingPrice.update_layout(title_text="Tendency of Closing Price - Per Company")
tendencyClosingPrice.show()

- **Distribution & Tendency of Values - Sales Volume**

############ Add description of histogram and scatter plot per company, plot will be dynamic and you can filter per company

        - Company 1
        - Company 2
        - Company 3
        - Company 4
        - Company 5

In [14]:
# Checking distribution for Sales Volume
histSalesVolume = px.histogram(dailyData,
                   x='salesVolume',
                   color='company',
                   marginal='box')
histSalesVolume.update_layout(title_text="Distribution of Sales Volume - Per Company")
histSalesVolume.show()

In [15]:
# Checking tendency for Sales Volume
tendencySalesVolume = px.scatter(dailyData,
                   x='day',y='salesVolume',
                   color='company')
tendencySalesVolume.update_layout(title_text="Tendency of Sales Volume - Per Company")
tendencySalesVolume.show()

- **Distribution & Tendency of Values - News Sentiment Index**

############ Add description of histogram and scatter plot per company, plot will be dynamic and you can filter per company.
Though it is not necesasry to analyze per comppany because all of them are using the same values. That is one of the disadvantages of working with the same value (Too many null values from the API)
Just describe general pattern.

In [16]:
# Checking distribution for News Sentiment Index
histSentimentIndex = px.histogram(dailyData,
                   x='News Sentiment',
                   color='company',
                   marginal='box')
histSentimentIndex.update_layout(title_text="Distribution of News Sentiment Index - Per Company")
histSentimentIndex.show()

In [17]:
# Checking tendency for News Sentiment Index
tendencySentimentIndex = px.scatter(dailyData,
                   x='day',y='News Sentiment',
                   color='company')
tendencySentimentIndex.update_layout(title_text="Tendency of News Sentiment Index - Per Company")
tendencySentimentIndex.show()

#### b) Multivariate Analysis

- **Scatter Plots Matrix for Numerical Variables**

In order to visualize better the relationship between variables, we will temprarily standardize the values for our variales per company.

In [18]:
# Library for Standardization
from sklearn.preprocessing import StandardScaler 

In [59]:
# Grouoing data per company
grouped = dailyData.groupby('company')
# Creating empty dataframe
standardizedDailyData = pd.DataFrame()
# Standardizing Values
scaler = StandardScaler()
for group, group_df in grouped:
    group_df_std = group_df.copy()
    group_df_std[['closingPrice','salesVolume','News Sentiment']] = scaler.fit_transform(group_df_std[['closingPrice','salesVolume','News Sentiment']])
    standardizedDailyData = pd.concat([standardizedDailyData,group_df_std])
# Adding column with company
standardizedDailyData.reset_index(drop = True, inplace = True)


Describe scatter plot matrix here: association between variables, per company

        - ASST
        - GCT
        - KSCP
        - MGAM
        - SNAL

In [60]:
# Creating scatter plot matrix
scatterMatrix = px.scatter_matrix(standardizedDailyData,
    dimensions=['closingPrice','salesVolume','News Sentiment'],
    color="company")
scatterMatrix.show()

## III. Data Pre-Processing & Modeling

#### Autoregressive-moving-average (ARMA) Model

##### a) Initial Transformation of Values

For this model, we will use the % Change in Share Price.

- **Creating new dataframe**

In [66]:
# Creating dataframe with day, closingPrice and Company
ARMAdata = dailyData[['day','closingPrice','company']]
ARMAdata.head(n = 5)

,day,closingPrice,company
0,2023-10-13,8.69,GCT
1,2023-10-12,9.19,GCT
2,2023-10-11,8.77,GCT
3,2023-10-10,9.07,GCT
4,2023-10-09,9.96,GCT


- **Defining company as a category**

In [67]:
# Defining company as categorical vraible
ARMAdata['company'] = ARMAdata['company'].astype("category")
ARMAdata.dtypes

day             datetime64[ns]
closingPrice           float64
company               category
dtype: object

- **Sorting dataframe by company and day**: Previous step for calculation

In [68]:
# Sorting by company and then by day
ARMAdata = ARMAdata.sort_values(by = ['company','day'], ascending = [True,True])
ARMAdata.reset_index(drop = True, inplace = True)
ARMAdata.head(n = 5)

,day,closingPrice,company
0,2023-02-03,3.55,ASST
1,2023-02-06,2.55,ASST
2,2023-02-07,2.43,ASST
3,2023-02-08,2.20,ASST
4,2023-02-09,1.87,ASST


- **% Change Share Price**: Calculation of % Change

In [69]:
ARMAdata['percentChange'] = ARMAdata.groupby('company')['closingPrice'].pct_change()
ARMAdata.head(n = 5)

,day,closingPrice,company,percentChange
0,2023-02-03,3.55,ASST,NaN
1,2023-02-06,2.55,ASST,-0.281690
2,2023-02-07,2.43,ASST,-0.047059
3,2023-02-08,2.20,ASST,-0.094650
4,2023-02-09,1.87,ASST,-0.150000


In [70]:
ARMAdata.shape

(1434, 4)

- **Removing First day for each company**: We are removing the first day of data for each company because it is not possible to calculate the % change

In [71]:
ARMAdata.dropna(subset = ['percentChange'], inplace = True)
ARMAdata.head(n = 5)

,day,closingPrice,company,percentChange
1,2023-02-06,2.55,ASST,-0.281690
2,2023-02-07,2.43,ASST,-0.047059
3,2023-02-08,2.20,ASST,-0.094650
4,2023-02-09,1.87,ASST,-0.150000
5,2023-02-10,1.74,ASST,-0.069519


In [72]:
ARMAdata.shape

(1429, 4)

- **Normalization of Values**: Do not forget to explain why we are standardizing the values

In [73]:
# Grouoing data per company
groupedARMA = ARMAdata.groupby('company')
# Creating empty dataframe
standardizedPercentChange = pd.DataFrame()
# Standardizing Values
percentChangeScaler = StandardScaler()
for group, group_df in groupedARMA:
    group_df_std = group_df.copy()
    group_df_std[['percentChangeNormalized']] = percentChangeScaler.fit_transform(group_df_std[['percentChange']])
    standardizedPercentChange = pd.concat([standardizedPercentChange,group_df_std])
# Adding column with company
standardizedPercentChange.reset_index(drop = True, inplace = True)

In [75]:
standardizedPercentChange.head()

,day,closingPrice,company,percentChange,percentChangeNormalized
0,2023-02-06,2.55,ASST,-0.281690,-2.553272
1,2023-02-07,2.43,ASST,-0.047059,-0.383426
2,2023-02-08,2.20,ASST,-0.094650,-0.823546
3,2023-02-09,1.87,ASST,-0.150000,-1.335416
4,2023-02-10,1.74,ASST,-0.069519,-0.591133


##### b) Training / Test (70% - 30%) Datasets

In [80]:
# Library to split data
from sklearn.model_selection import train_test_split

In [81]:
# Creating empty dataframes
trainARMA = pd.DataFrame()
testARMA = pd.DataFrame()

# Splitting Training / Testing data
for company in standardizedPercentChange["company"].unique():

    companyData = standardizedPercentChange[standardizedPercentChange["company"] == company]
    
    trainSub, testSub = train_test_split(companyData, test_size=0.3, shuffle=False, stratify = None)
    
    trainARMA = pd.concat([trainARMA, trainSub], ignore_index=True)
    testARMA = pd.concat([testARMA, testSub], ignore_index=True)

In [84]:
trainARMA[trainARMA['company'] == 'ASST'].tail(n = 5)

,day,closingPrice,company,percentChange,percentChangeNormalized
116,2023-07-25,1.1800,ASST,-0.008403,-0.025944
117,2023-07-26,1.1598,ASST,-0.017119,-0.106542
118,2023-07-27,1.0900,ASST,-0.060183,-0.504795
119,2023-07-28,1.1500,ASST,0.055046,0.560828
120,2023-07-31,1.2100,ASST,0.052174,0.534268


In [85]:
testARMA[testARMA['company'] == 'ASST'].head(n = 5)

,day,closingPrice,company,percentChange,percentChangeNormalized
0,2023-08-01,1.10,ASST,-0.090909,-0.788949
1,2023-08-02,1.11,ASST,0.009091,0.135841
2,2023-08-03,1.12,ASST,0.009009,0.135083
3,2023-08-04,1.13,ASST,0.008929,0.134340
4,2023-08-07,1.06,ASST,-0.061947,-0.521109


##### d) Validating ARMA Assumptions 

For each company we need to test 3 assumptions: White noise, Stationarity, and Seasonnality.
For each assumption, we will use two approaches:

    - 1st, we will make a conclusion by checking patterns on a Visualization
    - 2nd, we will make a conclusion using a hypotesis testing: state null and alternative hypotesis, decide (Reject or fail to reject null) based on P-value

We know that 2 companies cannot be used for ARMA models

In [105]:
# Library to perform statistical tests
from statsmodels.tsa.stattools import adfuller, acf, pacf
# Library to manipulate data and perform calculations
import numpy as np

- **White Noise - Visual Approach**: Here we will decide if an ARMA modle is possible. If not, stop the analysis for the company. We will use the dynamic plot below: Auto Correlation Function (ACF): The Autocorrelation Function (ACF) from the statsmodels.tsa.stattools library is not a statistical test itself, but rather a statistical tool used to analyze and visualize autocorrelation in time series data. The ACF is a function that represents the correlation between a time series and its own lagged values at different time lags. It helps you understand the temporal dependencies within a time series. The ACF is not a hypothesis test, and it doesn't involve null or alternative hypotheses. 

For White Noise - Visual Approach: You need to check the following in the plot:

    - Constant variance: For the most part, our ticker percent changes have constant variance (suggesting we need to to further analysis). Refer to plot from exploratory analysis. If they are not there, be sure we add them to the EDA section
    - Mean: Is 0 (also suggesting we need to to further analysis namely looking at the ACF)

        - Company 1
        - Company 2
        - Company 3
        - Company 4
        - Company 5

In [188]:
# Empty Dictionaries
acfResults = {}
acfSignificantValues = {}

# Implementing ACF with different time lags
for company in trainARMA["company"].unique():
    temporary = trainARMA[trainARMA["company"] == company]
    acfResults[company] = acf(temporary["percentChangeNormalized"], nlags=20, fft=False)
    acfSignificantValues[company] = (1.65/np.sqrt(len(temporary["percentChangeNormalized"])))

# Creating dataframe with results per company
acfResults = pd.DataFrame(acfResults)
acfSignificantValues = pd.DataFrame(acfSignificantValues, index = [0])
# Removing fist row
acfResults.drop(0,inplace = True)
# Adding index as a column
acfResults['index'] = acfResults.index
# Creating dataframe for plot
acfResultsPlot = pd.melt(acfResults, id_vars = ['index'], value_vars = ['ASST','GCT','KSCP','MGAM','SNAL'],
                         var_name = 'company', value_name = 'acfResult')
# Creating dataframe with Significant values
acfSignificantValuesTable = pd.melt(acfSignificantValues, var_name = 'company', value_name = 'acfSignificantValue')
# Adding significant values to acfResults
acfPlot = acfResultsPlot.merge(acfSignificantValuesTable, left_on=['company'], right_on=['company'], how='left')
# Changing Name of New Column
acfPlot.rename(columns = {'acfSignificantValue':'upperSignificantValue'}, inplace = True)
# Adding lower significant value
acfPlot['lowerSignificantValue'] = -acfPlot['upperSignificantValue']


In [189]:
acfPlot.head()

,index,company,acfResult,upperSignificantValue,lowerSignificantValue
0,1,ASST,-0.234992,0.15,-0.15
1,2,ASST,0.187155,0.15,-0.15
2,3,ASST,0.032533,0.15,-0.15
3,4,ASST,0.070744,0.15,-0.15
4,5,ASST,0.002440,0.15,-0.15


In [191]:
# Checking ACF results
#acfResultsBar = px.line(x = acfPlot['index'],y = acfPlot['upperSignificantValue'], color = acfPlot['company'])
acfResultsBar = px.line(acfPlot,
                   x = 'index',y = ['acfResult','upperSignificantValue','lowerSignificantValue'],
                   color_discrete_sequence = ['blue','red','red'], facet_col = 'company')
acfResultsBar.update_layout(title_text="ACF Results for Percent Change Share Price Normalized - Per Company")
acfResultsBar.show()

- **Seasonality - Visual Approach**: We will use the dynamic plot below. 

Analyze:

    - Trend and Seasonality Analysis: Examine the time series data for any discernible trends or seasonality patterns. Trends can be identified through visual inspection of the data or by using techniques like moving averages. Seasonality refers to repetitive patterns that occur at fixed intervals, such as daily, weekly, or monthly. If significant trends or seasonality exist, additional preprocessing steps may be required, such as detrending or deseasonalizing the data.

        - Company 1
        - Company 2
        - Company 3
        - Company 4
        - Company 5

In [100]:
# Checking tendency for closing price
tendencyPercentChangeNormalized = px.scatter(trainARMA,
                   x='day',y='percentChangeNormalized',
                   color='company',
                   marginal_y = 'box')
tendencyPercentChangeNormalized.update_layout(title_text="Time Series Plot for Percent Change Share Price Normalized - Per Company")
tendencyPercentChangeNormalized.update_traces(row=1, col=1,mode='lines+markers', line_shape='linear')
tendencyPercentChangeNormalized.show()

- **Stationarity - Visual & Hypothesis Testing Approaches**: ADF test

For visual approach use Time series plot from previous section

ARMA models require the underlying time series data to be stationary. Stationarity means that the statistical properties of the data, such as mean, variance, and autocorrelation, remain constant over time. WE can perform a stationarity check by conducting statistical tests such as the Augmented Dickey-Fuller (ADF) test. If the data is non-stationary, it needs to be transformed or differenced to achieve stationarity (in the ARIMA model we would difference lags to flatten the data).

Define significance level at 0.05

H0: The time series is non-stationary. In other words, it has some time-dependent structure and does not have constant variance over time.

HA: The time series is stationary.

more references: https://www.statology.org/dickey-fuller-test-python/

Return of the function: https://docs.w3cub.com/statsmodels/generated/statsmodels.tsa.stattools.adfuller

    - adf (float) – Test statistic
    - pvalue (float) – MacKinnon’s approximate p-value based on MacKinnon (1994, 2010)
    - usedlag (int) – Number of lags used
    - nobs (int) – Number of observations used for the ADF regression and calculation of the critical values
    - critical values (dict) – Critical values for the test statistic at the 1 %, 5 %, and 10 % levels. Based on MacKinnon (2010)
    - icbest (float) – The maximized information criterion if autolag is not None.
    - resstore (ResultStore, optional) – A dummy class with results attached as attributes

In [211]:
# Empty Dictionaries
adfResults = {}
adfSignificantValues = {}

# Implementing ACF with different time lags
for company in trainARMA["company"].unique():
    temporary = trainARMA[trainARMA["company"] == company]
    adfResults[company] = adfuller(temporary["percentChangeNormalized"])
    adfSignificantValues[company] = (1.65/np.sqrt(len(temporary["percentChangeNormalized"])))

# Creating dataframe with results per company
adfResults = pd.DataFrame(adfResults)
adfSignificantValues = pd.DataFrame(adfSignificantValues, index = [0])

Below are the P-Values for each company and other test results 

First row: Test Statistic / Second Row: Critic Values for multiple significance levels / Third Row: Maximized AIC (Akaike Information Criterion)

Use all this statistics to make a conclusion: P-value, Test Statistic, Critic value. 

Let's double check how can we use the AIC metric.

In the context of the ADF test, a larger ICBest value suggests that more differencing is required to make the time series stationary. Conversely, a smaller ICBest value indicates that less differencing is needed.

The maximized information criterion helps determine the optimal order of differencing by selecting the d value that minimizes the criterion. This is important because selecting the wrong order of differencing can lead to incorrect modeling and forecasting.

In summary, the ICBest in the ADF test is used to find the optimal order of differencing that makes the time series stationary. A larger ICBest suggests more differencing is required, while a smaller ICBest indicates less differencing is needed. The goal is to select the order of differencing that provides the best fit for the model.

In [220]:
# Set the display option for scientific notation
pd.options.display.float_format = '{:.2e}'.format
# Showing P-Values per test
print(adfResults.iloc[1,])
# Reverting scientific notation change
pd.options.display.float_format = None


ASST   2.02e-26
GCT    0.00e+00
KSCP   1.64e-14
MGAM   8.63e-25
SNAL   2.24e-27
Name: 1, dtype: object


In [219]:
# Adjust the display option to show the entire content of a cell
pd.options.display.max_colwidth = None
# ADF test results
adfResults.iloc[[0,4,5]]

,ASST,GCT,KSCP,MGAM,SNAL
0,-14.16834,-31.136367,-8.843624,-13.254834,-14.782393
4,"{'1%': -3.486055829282407, '5%': -2.8859430324074076, '10%': -2.5797850694444446}","{'1%': -3.4631437906252636, '5%': -2.8759570379821047, '10%': -2.574454682874228}","{'1%': -3.452713099849546, '5%': -2.871387839457783, '10%': -2.5720171933352485}","{'1%': -3.46172743446274, '5%': -2.8753374677799957, '10%': -2.574124089081557}","{'1%': -3.4718957209472654, '5%': -2.8797795410156253, '10%': -2.5764947265625}"
5,314.563049,213.439189,390.40239,606.251095,406.70298


##### c) ARMA Model Selection

We will analyze 2 components to decide the type of model: Autoregression Component (PACF) and Moving Average Component (ACF)

Unlike the previous section, we will use a Visual approach to decide the best model. We need to explain the pattern of data poitns and describe the reasons why we are defining the values for the hyperparameters.

We need to define the degree (# of lags)

We wil complete this analysis for 3 companies:

Company 1:
- **Autoregression component**
- **Moving average component**

Company 2:
- **Autoregression component**
- **Moving average component**

Company 3:
- **Autoregression component**
- **Moving average component**



##### e) Training Phase

Per company, we should train our models.

Include crossvalidation and calculation of evaluation metrics (RMSE and Confusion Matrix)

Company 1:
- **Training**
- **Cross Validation**
- **Evaluation Metrics for Training**

Company 2:
- **Training**
- **Cross Validation**
- **Evaluation Metrics for Training**

Company 3:
- **Training**
- **Cross Validation**
- **Evaluation Metrics for Training**

##### f) Testing Phase

Per company, we should test our models.

Include calculation of evaluation metrics (RMSE and Confusion Matrix)

Company 1:
- **Testing**
- **Evaluation Metrics for Training**

Company 2:
- **Testing**
- **Evaluation Metrics for Training**

Company 3:
- **Testing**
- **Evaluation Metrics for Training**